<h1> Applied Data Science Capstone</h1>

<h1>Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto</h1>

This Assignment is based on the Lab: Segmenting and Clustering Neighborhoods in New York City.

The Assignment consists of 3 parts.

* Part 1 - obtain the Post codes of Canada and prepare a Pandas dataframe with one row for each post code providing the Borough and Neighborhood(s)
* Part 2 - Add the Latitude and Longitude of the area of the Post Code to the Pandas dataframe
* Part 3 - Explore, cluster and analyse the neighborhoods of Toronto

For those who have to grade this assignment: This Jupyter Notebook will be submitted 3 times. Please mark the Part 1 the first time, Part 2 the second time and Part 3 the third time. 

(I have used html to do the heading because the "#"'s worked sometimes - but mostly not. )

<h1> PART 1 </h1>

<h2>Source the Post code data of Canada and wrangle it into a Pandas dataframe </h2>

<h3> Preparation:</h3>
<h4>Do the software installs for Part 1 - only once per launch of notebook</h4>

In [1]:
!conda install -c conda-forge beautifulsoup4 --yes
print("INSTALLED BEAUTIFULSOUP4")
!conda install -c conda-forge lxml --yes
print("INSTALLED LXML")
!conda install -c conda-forge requests --yes
print("INSTALLED REQUESTS")

Solving environment: done

# All requested packages already installed.

INSTALLED BEAUTIFULSOUP4
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    lxml-4.4.1                 |   py36h7ec2d77_0         1.6 MB  conda-forge

The following packages will be UPDATED:

    lxml: 4.3.1-py36hefd8a0e_0 --> 4.4.1-py36h7ec2d77_0 conda-forge


lxml-4.4.1           | 1.6 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
INSTALLED LXML
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - requests


The following packages will be downloaded:

    package                    |            build
    ---------------

<h3>Do all the imports - with launch or when you have to restart the python kernel</h3>

In [2]:
from bs4 import BeautifulSoup
import requests

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print('IMPORTS DONE')


IMPORTS DONE


<h3>Import the Canadian Post codes table from the Wikipedia webpage utilising BeautifulSoup</h3>

In [3]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source,'lxml')
          
post_codes = soup.find('table',class_="wikitable sortable")

print('FOUND THE POST CODE TABLE ON WIKIPEDIA')

FOUND THE POST CODE TABLE ON WIKIPEDIA


<h4>Use BeautifulSoup find_all to extract the rows from the Canadian Postal code table</h4>

In [4]:
table_rows = post_codes.find_all('tr')

<h3>Prepare the empty Pandas Dataframe</h3>

In [5]:
# define the dataframe columns
column_names = ['PostalCode','Borough', 'Neighborhood'] 

# instantiate the dataframe
pd_post_codes = pd.DataFrame(columns=column_names)

print(pd_post_codes.columns)
pd_post_codes


Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')


,PostalCode,Borough,Neighborhood


<h3>Cameras, lighting, shift-enter</h3>

#### Loop through the table rows and add it to the pd_post_codes dataframe


In [6]:
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td] # contains a list
    if len(row)> 0:
        #Only non-empty rows will be added
        row_postalcode = row[0]
        if row[1] !=  'Not assigned': 
            # borough that is assigned
            row_borough = row[1]
            row_neighborhood = row[2] 
            #  - w.r.t. the "\n" - I am using it to put commmas in between the Neighborhoods. 
            if row_neighborhood == 'Not assigned\n':
                row_neighborhood = row_borough
            pd_post_codes = pd_post_codes.append({'PostalCode':row_postalcode,'Borough':row_borough, 'Neighborhood':row_neighborhood},ignore_index=True)

print('The rows should now be in pd_post_codes - albeit with possibly more than one row per post code') 



The rows should now be in pd_post_codes - albeit with possibly more than one row per post code


<h3>Replace the "\n" in the Neighborhood column with a comma so that there will be a comma between the neighbourhoods once concatenated.

In [7]:
pd_post_codes['Neighborhood'] = pd_post_codes['Neighborhood'].str.replace('\n',',') 

<h3>Combine neighborhoods with the same borough

In [8]:
pd_post_codes = pd_post_codes.groupby(['PostalCode','Borough'],as_index=False)['Neighborhood'].sum()

<h4> Remove the last character - which is the comma of the rightmost neighborhood in the list</h4>

In [10]:
pd_post_codes['Neighborhood']= pd_post_codes['Neighborhood'].str[:-1]

In [11]:
pd_post_codes.shape

(103, 3)

<h2> Part 2 </h2>

<h3> Read in the csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data</h3>
    

In [12]:
Postal_codes_coordinates='http://cocl.us/Geospatial_data'
    
pd_Postal_codes_coordinates = pd.read_csv(Postal_codes_coordinates)

<h3> Rename the 'Postal Codes'- column in order for the Merge to work. (I have tried it without renaming.)

In [13]:
pd_Postal_codes_coordinates.rename(columns={'Postal Code':'PostalCode'},inplace=True)


<h3>Merge the two dataframes using the PostalCode column to join.</h3>

In [14]:

pd_Postal_codes_part2 =pd.merge(pd_post_codes,pd_Postal_codes_coordinates[['PostalCode','Latitude','Longitude']],
left_on='PostalCode',
right_on='PostalCode',
how='outer')

pd_Postal_codes_part2.shape

(103, 5)

<h1>Part 3.Explore, cluster and analyse the neighborhoods in Toronto.</h1>

<h4> Working with only boroughs that contain the word Toronto and then replicate the same analysis as with the New York City data.

<h2>1.Preparation</h1>

<h3>Do the installs</h3>


In [15]:
!conda install -c conda-forge geopy --yes 
print('INSTALLED GEOPY')
!conda install -c conda-forge folium=0.5.0 --yes 
print('INSTALLED FOLIUM')
print('INSTALLATION DONE.')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.21.0-py_0 conda-forge


geographiclib-1.50   | 34 KB     | ##################################### | 100% 
geopy-1.21.0         | 58 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
INSTALLED GEOPY
Solving environment: done

## Packa

<h3> Do the imports</h3>

In [16]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from pandas.io.json import json_normalize

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import json # library to handle JSON files


import folium # map rendering library

print('LIBRARIES IMPORTED.')

LIBRARIES IMPORTED.


A Pandas DataFrame has been created for Toronto that is similar to the one for New York in Part2.

In [17]:
pd_Postal_codes_part2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malver",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Unio",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hil",43.763573,-79.188711
3,M1G,Scarborough,Wobur,43.770992,-79.216917
4,M1H,Scarborough,Cedarbra,43.773136,-79.239476


But we only want to work with Toronto.

In [18]:
pd_Postal_codes_Toronto=pd_Postal_codes_part2[pd_Postal_codes_part2['Borough'].str.contains('Toronto')]


In [19]:
print('Now we are only working with {} boroughs in Toronto.'.format(pd_Postal_codes_Toronto.shape[0]))
print('Here are the first 5 rows')
pd_Postal_codes_Toronto.head()


Now we are only working with 39 boroughs in Toronto.
Here are the first 5 rows


,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beache,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdal",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaa",43.668999,-79.315572
43,M4M,East Toronto,Studio Distric,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Par,43.728020,-79.388790


#### Use geopy library to get the latitude and longitude values of Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [20]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [21]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(pd_Postal_codes_Toronto['Latitude'], pd_Postal_codes_Toronto['Longitude'], pd_Postal_codes_Toronto['Borough'], pd_Postal_codes_Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  


map_Toronto

Please feel free to zoom in where the action is using the "+" in the tot left corner.

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version - Hidden

In [22]:
# The code was removed by Watson Studio for sharing.

#### Let's explore the first neighborhood in our dataframe.

I have picked an area in the West of Toronto, say "High Park"

In [23]:
pd_Postal_codes_Toronto[pd_Postal_codes_Toronto['Neighborhood'].str.contains('High Park')]


,PostalCode,Borough,Neighborhood,Latitude,Longitude
82,M6P,West Toronto,"High Park,The Junction Sout",43.661608,-79.464763


In [24]:
pd_High_Park=pd_Postal_codes_Toronto[pd_Postal_codes_Toronto['Neighborhood'].str.contains('High Park')]

neighborhood_name = pd_High_Park['Neighborhood'].values[0]
neighborhood_latitude = pd_High_Park['Latitude'].values[0]
neighborhood_longitude = pd_High_Park['Longitude'].values[0]

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name,
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))



Latitude and longitude values of High Park,The Junction Sout are 43.6616083, -79.46476329999999.


#### Now, let's get the top 100 venues that are in High Park within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [25]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

VERSION = '20200314'

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

Send the GET request and examine the resutls

In [26]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e6ea1639fcb92001b231b68'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'High Park',
  'headerFullLocation': 'High Park, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 24,
  'suggestedBounds': {'ne': {'lat': 43.6661083045, 'lng': -79.458554540355},
   'sw': {'lat': 43.657108295499995, 'lng': -79.47097205964498}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '540f2951498e7516839a7126',
       'name': 'nodo',
       'location': {'address': '2885\xa0Dundas\xa0St.\xa0West',
        'lat': 43.665303321627505,
        'lng': -79.46562068004896,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.6653033

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [27]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [28]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,nodo,Italian Restaurant,43.665303,-79.465621
1,Lithuania Park,Park,43.658667,-79.463038
2,Hole in the Wall,Bar,43.665296,-79.465118
3,Indie Alehouse,Gastropub,43.665475,-79.465290
4,famous last words,Speakeasy,43.665181,-79.468471


And how many venues were returned by Foursquare?

In [29]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

24 venues were returned by Foursquare.


<a id='item2'></a>

## 2. Explore Neighborhoods inToronto

#### Using the function that was created in the Segmenting and clustering  of New York neighborhoods is used to repeat the same process to all the neighborhoods in Toronto

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *Toronto_venues*.

In [31]:

toronto_venues = getNearbyVenues(names=pd_Postal_codes_Toronto['Neighborhood'],
                                   latitudes=pd_Postal_codes_Toronto['Latitude'],
                                   longitudes=pd_Postal_codes_Toronto['Longitude']
                                  )



The Beache
The Danforth West,Riverdal
The Beaches West,India Bazaa
Studio Distric
Lawrence Par
Davisville Nort
North Toronto Wes
Davisvill
Moore Park,Summerhill Eas
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill Wes
Rosedal
Cabbagetown,St. James Tow
Church and Wellesle
Harbourfron
Ryerson,Garden Distric
St. James Tow
Berczy Par
Central Bay Stree
Adelaide,King,Richmon
Harbourfront East,Toronto Islands,Union Statio
Design Exchange,Toronto Dominion Centr
Commerce Court,Victoria Hote
Roselaw
Forest Hill North,Forest Hill Wes
The Annex,North Midtown,Yorkvill
Harbord,University of Toront
Chinatown,Grange Park,Kensington Marke
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagar
Stn A PO Boxes 25 The Esplanad
First Canadian Place,Underground cit
Christi
Dovercourt Village,Dufferi
Little Portugal,Trinit
Brockton,Exhibition Place,Parkdale Villag
High Park,The Junction Sout
Parkdale,Roncesvalle
Runnymede,Swanse
Queen's Par
Business Reply M

#### Let's check the size of the resulting dataframe

In [32]:
print(toronto_venues.shape)
toronto_venues.head()

(1716, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beache,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beache,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beache,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beache,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdal",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


Let's check how many venues were returned for each neighborhood

In [33]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmon",100,100,100,100,100,100
Berczy Par,56,56,56,56,56,56
"Brockton,Exhibition Place,Parkdale Villag",24,24,24,24,24,24
Business Reply Mail Processing Centre 969 Easter,19,19,19,19,19,19
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagar",17,17,17,17,17,17
"Cabbagetown,St. James Tow",43,43,43,43,43,43
Central Bay Stree,80,80,80,80,80,80
"Chinatown,Grange Park,Kensington Marke",89,89,89,89,89,89
Christi,18,18,18,18,18,18


#### Let's find out how many unique categories can be curated from all the returned venues

In [34]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 238 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [35]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])

toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beache,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [36]:
toronto_onehot.shape

(1716, 238)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [37]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide,King,Richmon",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.030000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.01,0.020000,0.000000,0.000000,0.01,0.010000,0.010000,0.00,0.000000,0.040000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.070000,0.000000,0.000000,0.000000,0.00,0.01,0.000000,0.000000,0.020000,0.00,0.00

#### Let's confirm the new size

In [38]:
toronto_grouped.shape

(39, 238)

#### Let's print each neighborhood along with the top 5 most common venues

In [39]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    #print(temp)
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmon----
              venue  freq
0       Coffee Shop  0.07
1        Restaurant  0.05
2   Thai Restaurant  0.04
3              Café  0.04
4  Sushi Restaurant  0.03


----Berczy Par----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2          Café  0.04
3   Cheese Shop  0.04
4      Beer Bar  0.04


----Brockton,Exhibition Place,Parkdale Villag----
            venue  freq
0            Café  0.12
1          Bakery  0.08
2     Coffee Shop  0.08
3  Breakfast Spot  0.08
4         Stadium  0.04


----Business Reply Mail Processing Centre 969 Easter----
           venue  freq
0    Yoga Studio  0.05
1           Park  0.05
2     Skate Park  0.05
3        Brewery  0.05
4  Burrito Place  0.05


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagar----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3             Plane  0.06
4       Coffee Shop  0.06




#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']


# create columns according to number of top venues
columns = ['Neighborhood']


for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted9 = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted9['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted9.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted9.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmon",Coffee Shop,Restaurant,Café,Thai Restaurant,Steakhouse,Bar,Sushi Restaurant,Seafood Restaurant,Lounge,Cosmetics Shop
1,Berczy Par,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Farmers Market,Beer Bar,Café,Bakery,Restaurant,Eastern European Restaurant
2,"Brockton,Exhibition Place,Parkdale Villag",Café,Breakfast Spot,Bakery,Coffee Shop,Music Venue,Stadium,Bar,Intersection,Convenience Store,Italian Restaurant
3,Business Reply Mail Processing Centre 969 Easter,Yoga Studio,Comic Shop,Skate Park,Brewery,Restaurant,Burrito Place,Recording Studio,Pizza Place,Moving Target,Spa
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boutique,Bar,Rental Car Location,Plane,Coffee Shop,Boat or Ferry


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [42]:
# set number of clusters
kclusters = 9

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([7, 7, 7, 1, 1, 7, 7, 7, 8, 7], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [43]:
# add clustering labels

neighborhoods_venues_sorted9.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged9 = pd_Postal_codes_Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged9 = toronto_merged9.join(neighborhoods_venues_sorted9.set_index('Neighborhood'), on='Neighborhood')


toronto_merged9 #.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beache,43.676357,-79.293031,6,Trail,Health Food Store,Pub,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Women's Store,Deli / Bodega
41,M4K,East Toronto,"The Danforth West,Riverdal",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Frozen Yogurt Shop,Pub,Pizza Place,Lounge,Liquor Store
42,M4L,East Toronto,"The Beaches West,India Bazaa",43.668999,-79.315572,1,Pizza Place,Movie Theater,Fish & Chips Shop,Pub,Sushi Restaurant,Fast Food Restaurant,Italian Restaurant,Burrito Place,Steakhouse,Board Shop
43,M4M,East Toronto,Studio Distric,43.659526,-79.340923,7,Café,Coffee Shop,Bakery,Italian Restaurant,Brewery,American Restaurant,Yoga Studio,Bookstore,Sandwich Place,Cheese Shop
44,M4N,Central Toronto,Lawrence Par,43.728020,-79.388790,2,Park,Construction & Landscaping,Swim School,Bus Line,Dessert Shop,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
45,M4P,Central Toronto,Davisville Nort,43.712751,-79.390197,1,Park,Breakfast Spot,Hotel,Gym,Gym / Fitness Center,Food & Drink Shop,Sandwich Place,Department Store,Discount Store,Dim Sum Restaurant
46,M4R,Central Toronto,North Toronto Wes,43.715383,-79.405678,1,Clothing Store,Coffee Shop,Spa,Salon / Barbershop,Restaurant,Café,Chinese Restaurant,Pet Store,Yoga Studio,Bagel Shop
47,M4S,Central Toronto,Davisvill,43.704324,-79.388790,7,Sandwich Place,Dessert Shop,Pizza Place,Coffee Shop,Italian Restaurant,Gym,Café,Sushi Restaurant,Pharmacy,Brewery
48,M4T,Central Toronto,"Moore Park,Summerhill Eas",43.689574,-79.383160,3,Playground,Trail,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
49,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,7,Pub,Coffee Shop,Supermarket,Sushi Restaurant,Sports Bar,Fried Chicken Joint,Restaurant,Pizza Place,American Restaurant,Liquor Store


Finally, let's visualize the resulting clusters

In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged9['Latitude'], toronto_merged9['Longitude'], toronto_merged9['Neighborhood'], toronto_merged9['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

I first had 5 clusters, but most of the Neighborhoods were in 1 cluster which now constitutes Cluster 8 . So I have increased it to 9 to see if I can get some more variety.

#### Cluster 1 - Rosedale - Affluent Neighborhood

(Reddish in the middle) If I just look at the venues, I see lots of outdoor activity (Park, Playground, Trail in positions 1- 3 with Dog Run in pos 10 . This is followed by Restaurants. When zooming in on the map, I see a green belt at the side of Rosedale, with few roads. My hunch was confirmed with a quick look at Wikipedia that this is an affluent neighbourhood (4.5 - 5 times larger than the average for Canada and with houses more than 100 years old. 

In [45]:
toronto_merged9.loc[toronto_merged9['Cluster Labels'] == 0, toronto_merged9.columns[[2] + list(range(5, toronto_merged9.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Rosedal,0,Park,Playground,Trail,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


#### Cluster 2 - Suburbs with no particular venue

(Purple color all over town) Apart from the Airport and the Business Reply Mail Processing Centre, it seems like these are the suburbs of Toronto with the Boroughs of East Toronto and West Toronto. On the Western side , it seems like Parkdale, Roncesvalles have all the amenities and a "green area:  So does "The Beaches West and "Indian Bazaar" on the eastern side. 

In [46]:
toronto_merged9.loc[toronto_merged9['Cluster Labels'] == 1, toronto_merged9.columns[[2] + list(range(5, toronto_merged9.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,"The Danforth West,Riverdal",1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Frozen Yogurt Shop,Pub,Pizza Place,Lounge,Liquor Store
42,"The Beaches West,India Bazaa",1,Pizza Place,Movie Theater,Fish & Chips Shop,Pub,Sushi Restaurant,Fast Food Restaurant,Italian Restaurant,Burrito Place,Steakhouse,Board Shop
45,Davisville Nort,1,Park,Breakfast Spot,Hotel,Gym,Gym / Fitness Center,Food & Drink Shop,Sandwich Place,Department Store,Discount Store,Dim Sum Restaurant
46,North Toronto Wes,1,Clothing Store,Coffee Shop,Spa,Salon / Barbershop,Restaurant,Café,Chinese Restaurant,Pet Store,Yoga Studio,Bagel Shop
68,"CN Tower,Bathurst Quay,Island airport,Harbourf...",1,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boutique,Bar,Rental Car Location,Plane,Coffee Shop,Boat or Ferry
76,"Dovercourt Village,Dufferi",1,Bakery,Pharmacy,Park,Coffee Shop,Café,Middle Eastern Restaurant,Bar,Bank,Supermarket,Furniture / Home Store
83,"Parkdale,Roncesvalle",1,Gift Shop,Movie Theater,Breakfast Spot,Eastern European Restaurant,Italian Restaurant,Bar,Dog Run,Restaurant,Dessert Shop,Bookstore
87,Business Reply Mail Processing Centre 969 Easter,1,Yoga Studio,Comic Shop,Skate Park,Brewery,Restaurant,Burrito Place,Recording Studio,Pizza Place,Moving Target,Spa


#### Cluster 3 - Northern Suburb - Lawrence Park - "standard suburb" with Dumpling Restaurant and Donut Shop

(Medium blue, Far north on the map.) Lawrence park. Looks like a standard suburb with a Dumpling restaurant and a donut shop. As it is far out(relative to the Toronto post codes suburbs"), time spent commuting must be taken into account. (Bus line is the 4th most common venue)

In [47]:
toronto_merged9.loc[toronto_merged9['Cluster Labels'] == 2, toronto_merged9.columns[[2] + list(range(5, toronto_merged9.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Lawrence Par,2,Park,Construction & Landscaping,Swim School,Bus Line,Dessert Shop,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


#### Cluster 4 - Northern Suburb - Moor Park, Summerhill East  - "standard suburb" with Dumpling Restaurant and Donut Shop

(Light blue, in the middle of the map.) Just a little bit north of Rosedal are the well established neighborhoods of  "Moore Park, Summerhill East". 

In [48]:
toronto_merged9.loc[toronto_merged9['Cluster Labels'] == 3, toronto_merged9.columns[[2] + list(range(5, toronto_merged9.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,"Moore Park,Summerhill Eas",3,Playground,Trail,Women's Store,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


#### Cluster 5 -  Northern Suburb - Roselawn - "standard suburb" with Dumpling Restaurant and Donut Shop¶

(Turcoise, to the left of Lawrence Hill) A  suburb with Dumpling Restaurant, Donut shop and Doner Restaurant

In [49]:
toronto_merged9.loc[toronto_merged9['Cluster Labels'] == 4, toronto_merged9.columns[[2] + list(range(5, toronto_merged9.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Roselaw,4,Home Service,Garden,Women's Store,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


#### Cluster 6 - Northen Suburb - Forest Hill - Affluent Standard suburb with Dumpling Restaurant and Donut shop

(light green, just north of the city centre) From the venues I gather that it is another suburb with Dumpling Restaurant, Donut shop and Doner Restaurant. Wikipedia, tells me it is an affluent suburb with doctors, lawyers, celebrities as residents. 

In [50]:
toronto_merged9.loc[toronto_merged9['Cluster Labels'] == 5, toronto_merged9.columns[[2] + list(range(5, toronto_merged9.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,"Forest Hill North,Forest Hill Wes",5,Park,Jewelry Store,Trail,Sushi Restaurant,Women's Store,Department Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


#### Cluster 7 - The Beaches - Health Food and speciality Stores

(Light Green, almost the same colour as the park, to the east of "The Beaches West") The Food landscape is different here - Health Food Store, Pub, Dessert Shop, Diner, Dim Sum Restaurant. The Women's stores are probaby the "independant speciality stores" mentioned in Wikipedia (https://en.wikipedia.org/wiki/The_Beaches.) What does not transpire in the single line above is that there are 4 Beaches close by. 

In [51]:
toronto_merged9.loc[toronto_merged9['Cluster Labels'] == 6, toronto_merged9.columns[[2] + list(range(5, toronto_merged9.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,The Beache,6,Trail,Health Food Store,Pub,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Women's Store,Deli / Bodega


#### Cluster 8 - Mostly City Centre with places to eat and drink, a theatre or two and a museum.

(Orange, in the city Center) Here the clustering is spot on - with Coffee shops high on the priority list. There are a few Neighborhoods that are out of the city Centre; On the west side are "Rummeymede, Swansae" and "High park,The Junction South". On the east side are Studio District or Harbourfront. And to the north are Davisvile" and "Deer Park,Forest Hill SE,Rathnelly,South Hill,..." So if food is your thing, then these are your options for moving from the one side to the other side of town

In [52]:
toronto_merged9.loc[toronto_merged9['Cluster Labels'] == 7, toronto_merged9.columns[[2] + list(range(5, toronto_merged9.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,Studio Distric,7,Café,Coffee Shop,Bakery,Italian Restaurant,Brewery,American Restaurant,Yoga Studio,Bookstore,Sandwich Place,Cheese Shop
47,Davisvill,7,Sandwich Place,Dessert Shop,Pizza Place,Coffee Shop,Italian Restaurant,Gym,Café,Sushi Restaurant,Pharmacy,Brewery
49,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",7,Pub,Coffee Shop,Supermarket,Sushi Restaurant,Sports Bar,Fried Chicken Joint,Restaurant,Pizza Place,American Restaurant,Liquor Store
51,"Cabbagetown,St. James Tow",7,Coffee Shop,Restaurant,Pub,Pizza Place,Italian Restaurant,Café,Bakery,Bank,Snack Place,Chinese Restaurant
52,Church and Wellesle,7,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Men's Store,Gastropub,Fast Food Restaurant,Hotel,Bubble Tea Shop
53,Harbourfron,7,Coffee Shop,Bakery,Park,Café,Pub,Theater,Restaurant,Mexican Restaurant,Breakfast Spot,Farmers Market
54,"Ryerson,Garden Distric",7,Coffee Shop,Clothing Store,Japanese Restaurant,Café,Middle Eastern Restaurant,Theater,Diner,Pizza Place,Electronics Store,Plaza
55,St. James Tow,7,Coffee Shop,Café,Restaurant,Italian Restaurant,Hotel,Beer Bar,Cosmetics Shop,Bakery,Clothing Store,Breakfast Spot
56,Berczy Par,7,Coffee Shop,Cocktail Bar,Seafood Restaurant,Cheese Shop,Farmers Market,Beer Bar,Café,Bakery,Restaurant,Eastern European Restaurant
57,Central Bay Stree,7,Coffee Shop,Italian Restaurant,Juice Bar,Burger Joint,Ice Cream Shop,Sandwich Place,Japanese Restaurant,Restaurant,Bubble Tea Shop,Café


#### Cluster 9 - Young Family

Inbetween the City Centre Neighborhoods is Christie. It does not look at all like the others with a grocery store in first position, a Candy store in 4th and a baby store in 6th. Add a nightclub to the mix. Young Family?

In [53]:
toronto_merged9.loc[toronto_merged9['Cluster Labels'] == 8, toronto_merged9.columns[[2] + list(range(5, toronto_merged9.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
75,Christi,8,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Baby Store,Gas Station,Nightclub,Coffee Shop,Restaurant


## 6. Analysis

To Conclude;
In Cluster 8 we have the City Centre Neighbourhoods with Coffee shops and the eating places. Although most are clustered in the ciy Centre, there are one or two out of town in every direction. On the west side are "Rummeymede, Swansae" and "High park,The Junction South". On the east side are Studio District or Harbourfront. And to the north are Davisvile" and "Deer Park,Forest Hill SE,Rathnelly,South Hill, So if our job seeker like his/her cofee and food, he can take the job.

In Cluster 2 we have Neighborhoods with no particular value spread over the whole area. They have all the amenities and parks.

In Clusters 3,4,5 we have - what I call the "standard neighborhoods" all to the north of the City with Dumpling Restaurants and Donut shops.

In Cluster 6, we have an affluent "standard neighborhood" with doctors, lawyers and celebrities - and Dumpling Restaurants and Donut shops. 

In Cluster 7, we have "The Beaches"-neighborhood with diffent kinds of food - Health shops and Dim Sum Restaurants. The woman's stores are most probably "Independant stores. And what the Venues do not tell, is that it is close to 4 beaches.

In Cluster 1, is a very affluent neighbourhood, Rosedale, with large plots, few roads and 100 year old houses. It borders a green belt. And that is why the venues are parks, playgrounds and trails.

In Cluster 9, which I call "Young Family", we have Grocery store in first place, a Candy store in 4th and a baby store in 6th. Definitely in a cla..uster of its own.

If our job seeker likes dumplings and Donuts, he/she can have his/her pick from the Northen Suburbs in Clusters 2-6.
If he/she likes coffee, the job seeker can pick and choose from east to west.
If he/she likes to feast on health food and at Dim Sum restaurants and going to the beach, there is only one choice - the neighbourhood "The Beaches"
And if he is part of young family, there is one neighborhood , Christies. 

### References

This notebook is part of a course on **Coursera** called *Applied Data Science Capstone*. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week3_LAB2).

<hr>

Copyright &copy; 2018 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).